<a href="https://colab.research.google.com/github/Jishan-works/Sentiment-analysis-using-DistillBERT-and-ktrain/blob/master/distillbert_ktrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 51.5MB/s 
     |████████████████████████████████| 421.8MB 23kB/s 
     |████████████████████████████████| 6.7MB 30.6MB/s 
     |████████████████████████████████| 983kB 13.1MB/s 
     |████████████████████████████████| 245kB 43.0MB/s 
     |████████████████████████████████| 778kB 24.7MB/s 
     |████████████████████████████████| 471kB 36.8MB/s 
     |████████████████████████████████| 276kB 41.4MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 3.9MB 32.3MB/s 
     |████████████████████████████████| 450kB 40.2MB/s 
     |████████████████████████████████| 1.1MB 36.9MB/s 
     |████████████████████████████████| 890kB 34.1MB/s 
     |████████████████████████████████| 3.0MB 32.1MB/s 
  Created wheel for ktrain: filename=ktrain-0.19.6-cp36-none-any.whl size=25268110 sha256=beb8adebe7fd7305ca5366ede9586f59b6a401ce611db109eb2be0075b4da98b
  Stored in directory: /root/.cache/pip/wheels/e4/a0/10/90232f

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [5]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf
tf.__version__

'2.1.0'

In [6]:
data_test = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx', dtype= str)
data_train = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx', dtype = str)

In [7]:
data_train.sample(5)

,Reviews,Sentiment
20890,"Wow, what can I say about this film? It's a lo...",neg
23789,"The 1st season was amazing, the whole idea of ...",pos
4495,This is a great movie from the lost age of rea...,pos
12996,I couldn't believe it. I had to rub my eyes a ...,neg
200,"Oh, man! This thing scared the heck out of me ...",neg


In [8]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face [https://arxiv.org/abs/1910.01108]


In [9]:
(train, val, preproc) = text.texts_from_df(train_df=data_train, text_column='Reviews', label_columns='Sentiment',
                   val_df = data_test,
                   maxlen = 400,
                   preprocess_mode = 'distilbert')

preprocessing train...
language: en
train sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


In [10]:
model = text.text_classifier(name = 'distilbert', train_data = train, preproc=preproc)

Is Multi-Label? False
maxlen is 400



done.


In [11]:
learner = ktrain.get_learner(model = model,
                             train_data = train,
                             val_data = val,
                             batch_size = 6)

In [12]:
learner.fit_onecycle(lr = 2e-5, epochs=2)



begin training using onecycle policy with max lr of 2e-05...
Train for 4167 steps, validate for 782 steps
Epoch 1/2
4167/4167 [==============================] - 3029s 727ms/step - loss: 0.2854 - accuracy: 0.8816 - val_loss: 0.1655 - val_accuracy: 0.9412
Epoch 2/2
4167/4167 [==============================] - 3013s 723ms/step - loss: 0.1587 - accuracy: 0.9414 - val_loss: 0.0633 - val_accuracy: 0.9842


In [13]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [14]:
predictor.save('/content/drive/My Drive/projects/DistillBert_ktrain')

In [21]:
data = ['I like her']

In [22]:
predictor.predict(data, return_proba=True)

array([[0.01607376, 0.98392624]], dtype=float32)

In [19]:
data = ['I will die for my country']

In [20]:
predictor.predict(data, return_proba=True)

array([[0.08347354, 0.91652644]], dtype=float32)

In [23]:
data = ['He is a mean person with mean attitude']

In [24]:
predictor.predict(data, return_proba=True)

array([[0.73907137, 0.26092863]], dtype=float32)